In [1]:
import pymc as pm
import numpy as np
import arviz as az

%load_ext lab_black
%load_ext watermark

# Lister

Adapted from [Unit 10: lister.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/lister.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/r.txt).

Associated lecture video: Unit 10 lesson 1

## Problem statement

Sir Joseph Lister (1827--1912), Professor of Surgery at Glasgow University, influenced by Pasteur's ideas, found that a wound wrapped in bandages treated by Carbolic acid (phenol) would often not become infected.

Here are Lister's data on treating open fractures and amputations:

Period    Carbolic acid used    Results

1864--1866     No       Treated   34 patients, 15 died and 19 recovered

1867--1870     Yes     Treated   40 patients,   6 died and 34 recovered


Estimate and interpret the risk difference,  risk ratio, and   odds ratio


## Notes

- Wishart! I've tried both WishartBartlett and LKJ, can't get a similar result to the professor's yet.
- possibly useful paper:
    - https://arxiv.org/pdf/1809.04746.pdf


Unfinished! This is not working correctly right now, assuming because of the different tau prior.

In [2]:
n1 = 34
y1 = 15
n2 = 40
y2 = 6
mu0 = np.array([-0.5, -1.5])
S = np.array([[0.1, 0], [0, 0.1]])

In [3]:
with pm.Model() as m:
    tau = pm.WishartBartlett("tau", S=S, nu=8)
    sigma = pm.Deterministic("sigma", pm.math.matrix_inverse(tau))
    mu = pm.MvNormal("mu", mu=mu0, cov=tau, shape=2)

    p1 = mu[0]
    p2 = mu[1]

    pm.Binomial("y1", logit_p=p1, n=n1, observed=y1)
    pm.Binomial("y2", logit_p=p2, n=n2, observed=y2)

    pm.Deterministic("rd", p1 - p2)
    pm.Deterministic("rr", p1 / p2)
    pm.Deterministic("or", (p1 / (1 - p1)) / (p2 / (1 - p2)))

    trace = pm.sample(3000)

Added new variable tau_c to model diagonal of Wishart.
Added new variable tau_z to model off-diagonals of Wishart.
/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/multipledispatch/dispatcher.py:27: AmbiguityWarning: 
Ambiguities exist in dispatched function _unify

The following signatures may result in ambiguous behavior:
	[ConstrainedVar, Var, Mapping], [object, ConstrainedVar, Mapping]
	[object, ConstrainedVar, Mapping], [ConstrainedVar, object, Mapping]
	[ConstrainedVar, object, Mapping], [object, ConstrainedVar, Mapping]
	[ConstrainedVar, Var, Mapping], [object, ConstrainedVar, Mapping]


Consider making the following additions:

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)
  warn(warning_text(dispatcher.name, ambiguities), AmbiguityWarnin

Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 56 seconds.


In [4]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
tau_z[0],-0.024,0.979,-1.817,1.836,0.010,0.009,10389.0,8397.0,1.0
mu[0],-0.315,0.303,-0.892,0.252,0.003,0.002,10089.0,8621.0,1.0
mu[1],-1.676,0.357,-2.375,-1.036,0.004,0.003,9791.0,8006.0,1.0
tau_c[0],5.368,3.166,0.612,11.142,0.028,0.021,11607.0,7042.0,1.0
tau_c[1],4.446,2.902,0.339,9.803,0.026,0.019,9364.0,5255.0,1.0
"tau[0, 0]",0.537,0.317,0.061,1.114,0.003,0.002,11607.0,7042.0,1.0
"tau[0, 1]",-0.005,0.229,-0.435,0.443,0.002,0.002,10130.0,8368.0,1.0
"tau[1, 0]",-0.005,0.229,-0.435,0.443,0.002,0.002,10130.0,8368.0,1.0
"tau[1, 1]",0.541,0.322,0.064,1.131,0.003,0.002,9458.0,6711.0,1.0
"sigma[0, 0]",3.735,3.753,0.518,9.382,0.047,0.034,10270.0,6860.0,1.0


In [5]:
%watermark -n -u -v -iv -p aesara,aeppl

Last updated: Fri Feb 03 2023

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.9.0

aesara: 2.8.10
aeppl : 0.1.1

pymc : 5.0.1
arviz: 0.14.0
numpy: 1.24.1

